# ALFABANK CAMPUS - Card transactions prediction

Одним из самых ценных источников информации о клиенте являются данные о банковских транзакциях. В этом соревновании участникам предлагается предсказать будущие траты клиента, используя информацию о совершенных тратах.
Данные представлены в виде массивов MCC кодов.

## Импорты и загрузка данных

In [1]:
import numpy as np
import pandas as pd

import math
import statistics
from scipy import linalg

from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [2]:
try:
    df_train = pd.read_csv('/home/aart/datasets/alfa/df_train.csv', sep=';')
    df_test = pd.read_csv('/home/aart/datasets/alfa/df_test.csv', sep=';')
except:
    url_test = "https://raw.githubusercontent.com/MarkvsLoopvs/alfa-campus/main/df_test.csv"
    url_train = "https://raw.githubusercontent.com/MarkvsLoopvs/alfa-campus/main/df_train.csv" 
    #в репозитории находятся только csv файлы, кода и ноутбуков там нет
    df_train = pd.read_csv(url_train, sep=';')
    df_test = pd.read_csv(url_test, sep=';')

In [3]:
df_train['Data'] = df_train.Data.apply(lambda s: list(map(int, s.split(','))))
df_train['Target'] = df_train.Target.apply(lambda s: list(map(int, s.split(','))))
df_test['Data'] = df_test.Data.apply(lambda s: list(map(int, s.split(','))))

## Функции метрик

In [4]:
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

## EDA

### Базовая информация

In [5]:
display(df_train.head(), df_test.head())


,Id,Data,Target
0,0,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 481...","[4814, 4814, 4814, 4814, 5411, 4814, 4814, 481..."
1,1,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 481...","[4814, 6011, 4814, 6011, 4814, 4814, 6011, 481..."
2,2,"[8021, 6011, 6011, 6010, 4829, 4814, 6011, 601...","[6011, 6011, 6010, 4829, 4829, 6010, 6011, 601..."
3,3,"[4814, 6011, 4814, 4814, 4814, 6011, 6011, 569...","[6011, 6011, 6010, 6011, 6011, 4814, 4814, 601..."
4,4,"[4814, 4814, 4814, 4814, 4814, 4814, 5946, 481...","[5499, 6011, 4814, 4829, 5200, 5411, 5499, 591..."


,Id,Data
0,0,"[4814, 4814, 6011, 6011, 6010, 6011, 6011, 481..."
1,1,"[6010, 6011, 6010, 5411, 5411, 5977, 6011, 601..."
2,2,"[4814, 6011, 5251, 6011, 7832, 5641, 5814, 482..."
3,3,"[6011, 4722, 4722, 4722, 4814, 6011, 6011, 482..."
4,4,"[4814, 4814, 4814, 6011, 4814, 4814, 4814, 481..."


In [6]:
display(df_train.shape, df_test.shape)


(7033, 3)

(7033, 2)

In [7]:
display(df_train.dtypes, df_test.dtypes)

Id         int64
Data      object
Target    object
dtype: object

Id       int64
Data    object
dtype: object

### Длины массивов

In [8]:
lengths = []
for i in range(len(df_train)):
    lengths.append(len(df_train.Data[i]))

In [9]:
print('Кратчайший массив в столбце df_train.Data:', min(lengths))
print('Длиннейший массив в столбце df_train.Data:', max(lengths))
print('Средняя длинна массива в столбце df_train.Data:', np.mean(lengths))
print('Медианная длина массива массива в столбце df_train.Data:', np.median(lengths))

Кратчайший массив в столбце df_train.Data: 40
Длиннейший массив в столбце df_train.Data: 21101
Средняя длинна массива в столбце df_train.Data: 473.3229062988767
Медианная длина массива массива в столбце df_train.Data: 336.0


Наблюдаем, что данные для обучения крайне неоднородны.

In [10]:
lengths_test = []
for i in range(len(df_test)):
    lengths_test.append(len(df_test.Data[i]))

In [11]:
print('Кратчайший массив в столбце df_test.Data:', min(lengths_test))
print('Длиннейший массив в столбце df_test.Data:', max(lengths_test))
print('Средняя длинна массива в столбце df_test.Data:', np.mean(lengths_test))
print('Медианная длина массива массива в столбце df_test.Data:', np.median(lengths_test))

Кратчайший массив в столбце df_test.Data: 40
Длиннейший массив в столбце df_test.Data: 88771
Средняя длинна массива в столбце df_test.Data: 476.7561495805488
Медианная длина массива массива в столбце df_test.Data: 344.0


Ситуация в тестовом датафрейме не лучше. 

### МСС-коды

In [12]:
train_codes = df_train.Data.explode().value_counts()
train_codes.head()

6011    700677
6010    490602
4814    473396
5411    472408
4829    307388
Name: Data, dtype: int64

In [13]:
print('Уникальных кодов в df_train.Data:', len(train_codes))

Уникальных кодов в df_train.Data: 184


In [14]:
test_codes = df_test.Data.explode().value_counts()
test_codes.head()

6011    694281
6010    519030
4814    483439
5411    467328
4829    304314
Name: Data, dtype: int64

In [15]:
print('Уникальных кодов в df_test.Data:', len(test_codes))

Уникальных кодов в df_test.Data: 184


Тренировочный и тестовый датафрейм делят множество MCC кодов. Топ-5 популярнейших кодов также совпадает.

## Первый подход - цепи Маркова

Первая идея пришедная мне на ум - использовать цепи Маркова. Assumption был следущий - пользователи в целом должны следовать какой-то рутине - снял деньги в банкомате, купил продуктов, заправил машину, etc. К тому же, рутина распространяется не только на последовательности событий, но и на поле возможных трат. Другими словами, экзотические и необычные траты редки, следовательно пренебрегаемы, следовательно не должны сильно влиять на метрику. Чтобы в этом убедиться, напишем функцию для вычислений переходных матриц.

In [16]:
def transition_matrix(transitions):
    n = 1+ max(transitions) # количество состояний (в данном случае - уникальных MCC-кодов)
    M = np.zeros((n,n)) # инициализация матрицы 
    
    # счетчик переходов между состояними 
    for (i,j) in zip(transitions,transitions[1:]):
        M[i,j] += 1

    # перевод в вероятности
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

Видим в переходной матрице некоторые довольно большие значения. Попробуем делать на их основе предсказания (цепи)

In [17]:
mchains=[]

for i in range(len(df_train)):
    df_train.Data[i], label = pd.factorize(df_train.Data[i])  
    m = transition_matrix(df_train.Data[i])
    
    num_iterations = 10
    # initialize the Markov Chain
    mc = np.zeros(num_iterations).astype(int)
    mc[0] = df_train.Data[i][-1]

    # iterate to generate the markov chain
    for j in range(1,num_iterations):
        mc[j] = np.argmax(np.random.multinomial(1,m[mc[j-1],:]))
    
    
    df_train.Data[i] = label[df_train.Data[i]]
    mchains.append(label[mc])

In [18]:
for i in range(len(mchains)):
    mchains[i] = list(mchains[i])
    
df_train['Predicted'] = mchains

In [19]:
print('Таргет по первому массиву:', df_train['Target'][0])
print('Предсказание по первому массиву:', df_train['Predicted'][0])

Таргет по первому массиву: [4814, 4814, 4814, 4814, 5411, 4814, 4814, 4814, 4814, 4814]
Предсказание по первому массиву: [4814, 5411, 6011, 6011, 6011, 5311, 4814, 6011, 4814, 6011]


In [20]:
print('Таргет по второму массиву:', df_train['Target'][1])
print('Предсказание по второму массиву:', df_train['Predicted'][1])

Таргет по второму массиву: [4814, 6011, 4814, 6011, 4814, 4814, 6011, 4814, 6011, 4814]
Предсказание по второму массиву: [6011, 4814, 4814, 4814, 6011, 4814, 6011, 4814, 6011, 4814]


Выглядит сносно. Но как оказывается этот подход не бьет даже **первое наивное** решение из бейзлайна:

In [21]:
mapk(df_train.Target,df_train.Predicted)

0.1860911316040706

## Дальнейшие действия

После такого низкого результата при казалось бы довольно неплохих на вид предсказаниях сильно расстроили и озадачили меня. Я начал размышлять и искать информацию - в первую очередь в материалах приложенных к самому заданию. Вскоре дошли руки и до досконального чтения приложенной статьи на хабре. И после ее прочтения я задумался - метрики ap@k и map@k используются в задачах ранжирования - например в recommendation engine'ах и т.п. Но ведь данная задача задачей ранжирования не является! Из следующей [статьи](https://habr.com/ru/companies/vk/articles/461927/) на хабре:

"Что подразумевается под задачей ранжирования? Представим, что в обучающей выборке есть какое-то множество запросов, для которых известен порядок документов по релевантности. Например, вы знаете, какой документ самый релевантный, какой второй по релевантности и т.д. И вам нужно восстановить такой порядок для всей генеральной совокупности. То есть для всех запросов из генеральной совокупности на первое место поставить самый релевантный документ, а на последнее — самый нерелевантный."

Но ведь нам нужно предсказать совсем не это, а последовательность последующих 10 mcc-кодов. Эти коды:
* а) Могут повторяться, и скорее всего будут это делать.
* б) Остаются релевантными постоянными. После того как пользователь купит продукты в магазине, это вовсе не означает что пользователь никогда не купит продукты в магазине.

Но в чем же именно проблема? Говорить про некорректность метрики можно сколько угодно, но что же приводит к низким результатам казалось бы неплохих решений? На мой взгляд, к этому приводит однин if-стейтмент из расчета  ap@k:

* for i, p in enumerate(predicted):
* if p in actual **and p not in predicted[:i]:**
* num_hits += 1.0
* score += num_hits / (i+1.0)


Таким образом, метрика **никак не поощряет** корректно угаданные позиции mcc-кодов, если такой же код уже был указан раннее. Но если мы посмотрим на массивы в столбце Target, мы увидим что mcc-коды в нем повторяются **постоянно**. Это имеет катастрофические последствия для моделей, которые действительно ищут какие-то закономерности, вроде тех же цепей Маркова и некоторых других, которых я обсужу в конце ноутбука. Что же поощряет эта метрика? Как мне кажется - максимальное разнообразие предиктов. Чем разнообразнее предикты - тем больше вероятность, что вам повезет угадать уникальный код и увеличить метрику.

Подводя итоги, задача является скорее задачей **seq2seq**, и требует иных метрик для оценки результатов. Но раз в задаче требуется оптимизация под map@k, то так тому и быть. Моей следующей идеей было:

## Оптимизация бейзлайна

Решение бейзлайн-2 выдает весьма неплохие результаты, по-крайней мере по сравнению с имплементированными мной цепями Маркова. Так же во время активной фазы решения задачи активно обсуждался следующий результат вызова функции map@k:

In [22]:
mapk(df_train.Target, df_train.Target)

0.3754930723415012

Таким образом (и я понимаю что говорить так некорректно), можно взять этот результат вызова mapk за бенчмарк со 100№ "точностью", что значит что у второго бейзлайна "точность" 0.32/0.37 = 86%+!. Весьма впечатляет. Что можно сделать чтобы ее улучшить?

### Расширить выборку, по которым вычисляются популярные транзакции.

Я пробовал разные комбинации и пермутации - основывать данные только на df_train.Data, только на df_train.Target, только на df_test.Data, на их сочетаниях. Лучшие результаты для меня показала следующая выборка: 

In [23]:
improved_sample = pd.concat([df_train['Data'], df_test['Data']])
improved_top10 = improved_sample.explode().value_counts().head(10)
improved_top10

6011    1394958
6010    1009632
4814     956835
5411     939736
4829     611702
5499     328357
5541     138904
5912     130783
5331     125113
5812     105092
Name: Data, dtype: int64

Также функция из бейзлайн 2 вычисляет самые популярные транзакции отдельных пользователей по всему массиву их исторических данных. С этим я тоже решил поэкспериментировать, и успешно. Как мы увидели на этапах EDA, длиннейший массив в тренировочных данных составляет 21101 позиций, а в тестовых данных - **88771**. Логично предположить, что для таких огромных массивов данных, паттерны поведения пользоваталей меняются. Допустим, человек совершает 10 транзакций в день. Такая оценка мне кажется даже не очень консервативной. Тогда данные до 88771 транзакций соответствуют 88771/10/365 = 24+ **лет** истории транзакций. Очевидно, что привычки у человека за такой срок поменяются. Так что мной было решено брать данные не по всем транзакциям, а последним нескольким *N* транзакциям. Мне кажется такой вывод был оправдан, и он улучшил мой результат. Еще одной идеей было поиграться с другим параметром функции - drop_from. Он отвечает за своеобразный "порог вхождения" в топ популярных транзакций пользователя. И, разумеется не стоит забывать вывод из предыдушего пункта - а именно, нам нужно поощрять максимальное разнообразие. Как этого достичь? Не допускать повторений. Функция из бейзлайна-2 может замешать к нам в предикты коды, которые уже были угаданы, если множество "самые популярные транзакции пользователя" и множество "самые популярные транзакции вообще" пересекаются. Условие на неповторение прописывается простым if-стейтментом. 
Таким образом, моя финальная функция выглядела так.

In [32]:
optimal_last = 0
optimal_drop_from = 0

def get_top_codes_improved(transactions, last=optimal_last, top_n=10, drop_from=optimal_drop_from):
    transactions_stats = sorted(
        Counter(transactions[-last:]).items(),
        key=lambda x: x[1], 
        reverse=True
    )[:top_n] #здесь все по старому, кроме использования последних last = optimal_last транзакций
    

    top_codes = [mcc_code for (mcc_code, count) in transactions_stats if count >= drop_from]
    
    
    i = 0
    while len(top_codes) < 10:
        if improved_top10.index[i] not in top_codes: 
            top_codes.append(improved_top10.index[i])
        i += 1                              #простой while цикл и if-statement следящий за разнообразием
    
    return top_codes[:10]

Найти лучшие oprimal_last и optimal_drop_from можно просто циклами. Мне нравится думать, что это примитивный gridsearch.

In [36]:
#best_mapk=0
#best_opt_l=0
#best_opt_drfr=0

#for optimal_last in range(1,1000):
#    for optimal_drop_from in range(1,11):
#        df_train['Predicted'] = df_train['Data'].apply(get_top_codes_improved)
#        if (mapk(df_train['Target'], df_train['Predicted']))>best_mapk:
#            optimal_last=best_opt_l
#            optimal_drop_from=best_opt_drfr

Код закомментирован, так как вложенный цикл исполняется довольно медленно. Но он полностью воспроизводим. Лучшие результаты полученные в итоге - optimal_last=195 и oprimal_drop_from=2

In [37]:
def get_top_codes_improved(transactions, last=195, top_n=10, drop_from=2):
    transactions_stats = sorted(
        Counter(transactions[-last:]).items(),
        key=lambda x: x[1], 
        reverse=True
    )[:top_n] 
    

    top_codes = [mcc_code for (mcc_code, count) in transactions_stats if count >= drop_from]
    
    
    i = 0
    while len(top_codes) < 10:
        if improved_top10.index[i] not in top_codes: 
            top_codes.append(improved_top10.index[i])
        i += 1                             
    
    return top_codes[:10]

In [38]:
df_train['Predicted'] = df_train['Data'].apply(get_top_codes_improved)
mapk(df_train['Target'], df_train['Predicted'])

0.33400263271335356

Результат mapk=0.334 на тренировочном датафрейме соответствует моему лучшему результату на лидерборде в 0.294

# Вывод

До сих пор убежден, что выбранная метрика некоррекнта для поставленной задачи seq2seq, и что все топовые решения по лидерборду в той или иной степени - оптимизация бейзлайн-2. Остаются два вопроса:
* Как бы я решал эту задачу на самом деле? Первым делом я бы имплементировал цепи Маркова, что и было сделано. Если бы решение не показало должных результатов, я бы рассматривал эту задачу как seq2seq. Следующими шагами было бы применение RNN - LTSM или GRU. Если бы и это не дало должных результатов - можно бы было токенизировать мсс-коды и решать эту задачу словно NLP - векторизация, трансформеры, и т.п.
* Какая метрика должна была бы быть использована? Я не знаю. Может быть какая-нибудь вариация MAPE, или вообще просто задефайнить precision/recall под эту задачу. 